<a id='section_0'></a>
# Рекомендация тарифов

Коммерческий департамент федерального оператора связи «Мегалайн» хочет понять, какой из двух тарифов - «Смарт» или «Ультра» - приносит больше денег.

В распоряжении - данные 500 пользователей компании: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018-й год.

Описание тарифов:

|                      | «Смарт»    |  «Ультра» |
|----------------------|------------|-----------|
|Ежемесячная плата     | 550 руб.   | 1950 руб. |
|Минут разговора       | 500 мин    | 3000 мин  |
|Сообщений             | 50         | 1000      |
|Интернет-трафик       | 15 Гб      | 30 Гб     |
|Стоимость услуг сверх тарифного пакета:        |
|Минута разговора*     | 3 руб.     | 1 руб.    |
|Сообщение             | 3 руб.     | 1 руб.    |
|1 Гб интернет-трафика*| 200 руб.   | 150 руб.  |

*Количество использованных минут и мегабайтов «Мегалайн» всегда округляет вверх. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.

*Цель исследования:* провести предварительный анализ тарифов на небольшой выборке клиентов и сделать вывод — какой тариф выгоднее.

*Этапы исследования:*

1. Чтение данных из предоставленного файла.
2. Подготовка данных
3. Построение моделей машинного обучения и оценка их качества:
  - RandomForest
  - LogisticRegression
  - DessisionTree
4. Проверка лучшей модели на тестовой выборке
5. Сравнение модели с константной
6. Итоговый вывод

## Откроем и изучим файл

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (mean_squared_error,
                             accuracy_score,
                             precision_score,
                             make_scorer)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

STATE = 1234

pd.options.mode.chained_assignment = None

In [ ]:
# Откроем файл с данными
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [ ]:
# Выведем информацию и датасете
print(f'Размерность исходных данных: строк - {df.shape[0]}, столбцов - {df.shape[1]}')
display(df.head())
df.info()

Размерность исходных данных: строк - 3214, столбцов - 5


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
# Посмотрим наличие явных дубликатов
df.duplicated().sum()

0

В данных не содержится пропусков и дубликатов. Данные чистые и готовы к исследованию

<a id='section_2'></a>
## Разделение данных на выборки

In [ ]:
# Определяем выборку с признаками и целевой признак
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

# Разделяем исходные данные на обучающую и тестовую выборки в соотношении 75:25
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    stratify=y,
    random_state=STATE)

print(f'Количество строк в обучающей выборке: {X_train.shape[0]}, или {(len(X_train) / len(df)):.1%} от исходных данных')
print(f'Количество строк в обучающей выборке: {X_test.shape[0]}, или {(len(X_test) / len(df)):.1%} от исходных данных')

Количество строк в обучающей выборке: 2410, или 75.0% от исходных данных
Количество строк в обучающей выборке: 804, или 25.0% от исходных данных


<a id='section_3'></a>
## Исследование моделей

**Построим прогноз с помощью модели RandomForestClassifier**

In [ ]:
rfr = RandomForestClassifier(random_state=STATE)

rfr_params={'max_depth': range(1, 11),
            'n_estimators': range(1, 11)}

grid_rfr = GridSearchCV(rfr,
                        rfr_params,
                        cv=3,
                        scoring=make_scorer(accuracy_score,
                                            greater_is_better=True))

grid_rfr.fit(X_train, y_train)

print(grid_rfr.best_params_)

model_rfr = grid_rfr.best_estimator_
model_rfr.fit(X_train, y_train)

predicted_rfr = model_rfr.predict(X_train)

print('Accuracy для модели RandomForestRegressor =', round(grid_rfr.best_score_, 3))

{'max_depth': 7, 'n_estimators': 9}
Accuracy для модели RandomForestRegressor = 0.807


**Построим прогноз с помощью модели LogisticRegression**

In [ ]:
lr = LogisticRegression(random_state=STATE)

lr_params={'max_iter': range(400, 901, 100)}

grid_lr = GridSearchCV(lr,
                       lr_params,
                       cv=3,
                       scoring=make_scorer(accuracy_score,
                       greater_is_better=True))

grid_lr.fit(X_train, y_train)

print(grid_lr.best_params_)

model_lr = grid_lr.best_estimator_
model_lr.fit(X_train, y_train)

predicted_lr = model_lr.predict(X_train)

print('Accuracy для модели LogisticRegression =', round(grid_lr.best_score_, 3))

{'max_iter': 400}
Accuracy для модели LogisticRegression = 0.734


**Построим прогноз с помощью модели "дерево решений"**

In [ ]:
dt = DecisionTreeClassifier(random_state=STATE,
                            criterion='gini')

dt_params={'max_depth': range(1, 11)}

grid_dt = GridSearchCV(dt,
                       dt_params,
                       cv=3,
                       scoring=make_scorer(accuracy_score,
                                           greater_is_better=True))

grid_dt.fit(X_train, y_train)

print(grid_dt.best_params_)

model_dt = grid_dt.best_estimator_
model_dt.fit(X_train, y_train)

predicted_dt = model_dt.predict(X_train)

print('Accuracy для модели DecisionTreeClassifier =', round(grid_dt.best_score_, 3))

{'max_depth': 3}
Accuracy для модели DecisionTreeClassifier = 0.793


**Промежуточный вывод:** Проведя исследования качества разных моделей получили следующие результаты:

- Наилучший результат показала модель, обученная алгоритмом "Случайный лес". Accuracy лучшей модели: 0.807
- Немного ниже результат у модели, обученной алгоритмом "Дерево решений". Accuracy лучшей модели: 0.734
- Значительно отстает модель, обученная по алгоритму "Логистическая регрессия". Accuracy модели: 0.793

<a id='section_4'></a>
## Проверьте модель на тестовой выборке

In [ ]:
# Построим новую модель по выбранным гиперпараметрам
model = RandomForestClassifier(
            random_state=STATE,
            n_estimators=9,
            max_depth = 7)

# Обучим модель на новой "большой" выборке
model.fit(X_train, y_train)

# Проверим модель на тестовой выборке
predications_test = model.predict(X_test)
test_result = accuracy_score(y_test, predications_test)
print(f'Accuracy на тестовой выборке: {round(test_result, 3)}')

Accuracy на тестовой выборке: 0.8


**Промежуточный вывод:** Accuracy, полученное на тестовой выборке с помощью модели, обученной по алгоритму "Случайный лес" равна 0.8055987558320373. Это хороший результат, если учитывать, что в задании требовалось довести показатель до уровня 0.75. Наш результат означает, что модель может предсказать правильный результат, примерно, в 80.6% случаев

<a id='section_5'></a>
## (бонус) Проверьте модели на адекватность

In [ ]:
# Проверим модель на адекватность
model_dummy = DummyClassifier(random_state=STATE, strategy='most_frequent')
model_dummy.fit(X_train, y_train)
result_dummy = model_dummy.score(X_test, y_test)
print(f'Accuracy равен: {round(result_dummy, 3)}')

Accuracy равен: 0.694


**Промежуточный вывод:** Accuracy, полученная моделью, обученной по алгоритму "Случайный лес" значительно выше, чем у модели алгоритма DummyClassifier. Таким образом, можно утверждать о вменяемости выбранного нами алгоритма.


Алгоритм DummyClassifier с параметром `strategy='most_frequent'` всегда возвращает наиболее часто используемую метку класса в наблюдаемом аргументе.

<a id='section_7'></a>
## Общий вывод

В данном проекте мы построили модель, на основе имеющихся данных о поведении клиентов, которые уже перешли на тарифы "Ультра" и "Смарт", которая выберет подходящий тариф для пользователей, использующих архивные тарифы.

1. В первой части работы мы произвели чтение данных из предоставленного файла и оценили качество полученных данных.
2. Во второй части, разделили исходные данные на обучающую и тестовую выборки в пропорции 75:25.
3. В третьей части мы построли три модели и обучили их на разных алгоритмах:
    - Построили прогноз с помощью модели "случайный лес". С помощью цикла подобрали оптимальные гиперпараметры глубины дерева и количества деревьев. При этом, метрика качества accuracy, для данной модели, составила 0.807
    - Построили прогноз с помощью модели "логистическая регрессия". Метрика accuracy, для данной модели, составила 0.734.
    - Построили прогноз с помощью модели "дерево решений". При помощи цикла, подобрали оптимальное значение глубины дерева, при котором метрика accuracy выдала самое большое значение 0.793, что оказалось немногим меньше значения, полученного моделью "случайный лес"
4. В четвертой части исследования мы произвели проверку качества выбранной модели "Случайный лес" на тестовой выборке и получили внушительный показатель accuracy, равный 0.800. Полученный показатель означает, что наша модель способна предсказать верный ответ в 80%.
5. В пятой части проверили нашу модель на вменяемость. Здесь мы сравнили значения accuracy, полученные нашей моделью и моделью, обученную алгоритмом DummyClassifier. Этот алгоритм, с параметром strategy='most_frequent' всегда возвращает наиболее часто используемую метку класса в наблюдаемом аргументе, то есть, приближен к модели, дающую случайные ответы.